# Prepare driver

In [1]:
! wget -P /home/jovyan https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar

--2020-02-29 11:02:28--  https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.84.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.84.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘/home/jovyan/postgresql-42.2.5.jar’

postgresql-42.2.5.j 100%[===================>] 806.58K  4.85MB/s    in 0.2s    

2020-02-29 11:02:29 (4.85 MB/s) - ‘/home/jovyan/postgresql-42.2.5.jar’ saved [825943/825943]



# Start spark application

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/jovyan/postgresql-42.2.5.jar --jars /home/jovyan/postgresql-42.2.5.jar pyspark-shell'

import pyspark

spark = pyspark.sql.SparkSession.builder \
        .master("local[1]") \
        .appName("snapshot") \
        .getOrCreate()

print("Application started")

Application started


# Warm-up spark

In [3]:
spark.sparkContext.range(1000).sum()
print("Spark application is ready for work")

Spark application is ready for work


# Read some PostgreSQL Data

In [5]:
customers = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='customers'
    ).load()

products = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='products'
    ).load()

orders = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='orders'
    ).load()

customers.registerTempTable("customers")
products.registerTempTable("products")
orders.registerTempTable("orders")

print("Customers table")
customers.show(5)
print("Orders table")
orders.show(5)
print("Products table")
products.show(10)


Customers table
+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1001|     Sally|   Thomas|sally.thomas@acme...|
|1002|    George|   Bailey|  gbailey@foobar.com|
|1003|    Edward|   Walker|       ed@walker.com|
|1004|      Anne|Kretchmar|  annek@noanswer.org|
+----+----------+---------+--------------------+

Orders table
+-----+----------+---------+--------+----------+
|   id|order_date|purchaser|quantity|product_id|
+-----+----------+---------+--------+----------+
|10001|2016-01-16|     1001|       1|       102|
|10002|2016-01-17|     1002|       2|       105|
|10003|2016-02-19|     1002|       2|       106|
|10004|2016-02-21|     1003|       1|       107|
+-----+----------+---------+--------+----------+

Products table
+---+------------------+--------------------+------+
| id|              name|         description|weight|
+---+------------------+--------------------+------+
|101|      

# Read and join the data

In [11]:
# Предыдущий вариант ДЗ
# query = """with total_weights as (
#            select o.id as order_id, o.purchaser as purchaser, o.quantity * p.weight as total_weight
#            from orders o left join products p on o.product_id = p.id
#            )
#            select c.id, c.first_name, c.last_name, sum(tw.total_weight) as total_weight, now() as load_dttm
#            from customers c 
#                 join total_weights tw on c.id = tw.purchaser
#            where c.id <= 1005
#            group by c.id, c.first_name, c.last_name"""

# Обновленный без with:
query = """
SELECT c.id, c.first_name, c.last_name, sum(o.quantity * p.weight) as total_weight, now() as load_dttm
FROM customers c
    LEFT JOIN orders o on c.id = o.purchaser
    LEFT JOIN products p on o.product_id = p.id
WHERE c.id <= 1005
GROUP BY c.id, c.first_name, c.last_name
HAVING total_weight is not null
"""

result = spark.sql(query)
result.write.mode("overwrite").format("parquet").save("/home/jovyan/weight_report")

In [12]:
spark.read \
    .format("parquet").load("/home/jovyan/weight_report").show()

+----+----------+---------+------------+--------------------+
|  id|first_name|last_name|total_weight|           load_dttm|
+----+----------+---------+------------+--------------------+
|1002|    George|   Bailey|        3.75|2020-02-29 11:24:...|
|1003|    Edward|   Walker|         5.3|2020-02-29 11:24:...|
|1001|     Sally|   Thomas|         8.1|2020-02-29 11:24:...|
+----+----------+---------+------------+--------------------+

